# Raport z analizy Tweet'ów

Celem [projektu](https://inclass.kaggle.com/c/angry-tweets) było zbudowanie klasyfikatora, pozwalającego na sklasyfikowanie Tweet'a jako pozytywnego, nutralnego, lub negatywnego. Klasyfikator zgodnie z założeniem został zbudowany w oparciu o [Python](https://www.continuum.io/downloads)'a oraz [scikit-learn](http://scikit-learn.org/stable/).

Niniejszy raport stanowi zwięzłe podsumowanie przeprowadzonych działań.

## 1. Wykorzystane bibilioteki

W projekcie wykorzystano następujące biblioteki:

In [ ]:
import pandas as pd
import re
import nltk
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

# 2. Przygotowanie Tweet'ów

Przed przystąpieniem do budowy klasyfikatora konieczne jest odpowiednie przygotowanie Tweet'ów.

## 2.1 Zdefiniowanie zmiennych

W poniższych zmiennych zostały zdefiniowane wyrażenia regularne oraz lista słów, które zostanły wykorzystane w dalszej części do przygotowania Tweet'ów.

In [ ]:
RE_SPACES = re.compile("\s+")
RE_HASHTAG = re.compile("[@#][_a-z0-9]+")
RE_HTTP = re.compile("http(s)?://[/\.a-z0-9]+")
RE_NUM = re.compile("[0-9]+")
RE_EMOTICONS = re.compile("(:-?\))|(:p)|(:d+)|(:-?\()|(:/)|(;-?\))|(<3)|(=\))|(\)-?:)|(:'\()|(8\))")

stopwords = ["a", "about", "after", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been",
            "before", "being", "between", "both", "by", "could", "did", "do", "does", "doing", "during", "each",
            "for", "from", "further", "had", "has", "have", "having", "he", "her", "here", "hers", "herself", "him",
            "himself", "his", "how", "i", "in", "into", "is", "it", "its", "itself", "let", "me", "more", "most", "my",
            "myself", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "own", "sha",
            "she", "should", "so", "some", "such", "than", "that", "the", "their", "theirs", "them", "themselves",
            "then", "there", "there's", "these", "they", "this", "those", "through", "to", "until", "up", "very",
            "was", "we", "were", "what", "when", "where", "which", "while", "who","whom", "with", "would", "you",
            "your", "yours", "yourself", "yourselves",
            "n't", "'s", "'ll", "'re", "'d", "'m", "'ve",
            "above", "again", "against", "below", "but", "cannot", "down", "few", "if",
             "over", "same", "too", "under", "why"]

## 2.2. Funkcje

Funkcja *normalize* umożliwia na zamianę encji html na poprawne znaki tekstowe.

In [ ]:
def normalize(text):
    text = text.strip().lower()
    text = text.replace('&nbsp;', ' ')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&amp;', '&')
    text = text.replace('&pound;', u'£')
    text = text.replace('&euro;', u'€')
    text = text.replace('&copy;', u'©')
    text = text.replace('&reg;', u'®')
    return text

Funkcja *tokenize* służy do tokenizacji, czyli wyodrębnianiu z tekstu poszczególnych słów i znaków interpunkcyjnych - tokenów.

In [ ]:
def tokenize(text):
    tokenizer = nltk.tokenize.TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

Funkcja *stem* służy do sprowadzania słów do ich podstawowej formy (*word stem*).

In [ ]:
def stem(tokens):
    stemmer = nltk.PorterStemmer()
    stems = []
    for token in tokens:
        stems.append(stemmer.stem(stem))
    return stems

Funkcja *clean_tokens* służy do usunięcia tokenów nie niosących dodatkowej treści tj. adresy url, hashtag'ów, odwołań do użtkowników, czy znaków interpunkcyjnych. Funkcja korzysta z wcześniej zdefiniowanych wyrażeń regularnych.

In [ ]:
def clean_tokens(tokens):
    i = 0
    while i < len(tokens):
        token = tokens[i]
        match = re.search(RE_HTTP, token) or re.search(RE_HASHTAG, token)
        match2 = (len(token) < 2) #or (token in string.punctuation)
        if (match is not None) or match2:
            del tokens[i]
        else:
            i += 1
    return tokens

Funkcja *remove_stopwords* pozwala na usunięcie tokenów, które znajdują się na liście *stopwords* - liście słów, które analogicznie jak powyżej nie stanowią istotnej treści.

In [ ]:
def remove_stopwords(words, stopwords):
    words_clean = [word for word in words if word not in stopwords]
    return words_clean

Zadaniem funkcji *preprocess* jest kompleksowe przygotowanie tweet'ów do dalszej analizy. Funkcja jest kompozycją wyżej przedstawionych kroków.

In [ ]:
def preprocess(tweets):
    tweets_tokenized = []

    for index, row in tweets.iterrows():
        tweet = row[-1]

        if not pd.isnull(tweet):
            tweet = normalize(tweet)
            tweet = tokenize(tweet)
            clean_tokens(tweet)
            tweet = remove_stopwords(tweet, stopwords)
        else:
            tweet = []

        if len(tweets.columns) == 3:
            tweets_tokenized.append([row[0], row[1], tweet])
        elif len(tweets.columns) == 2:
            tweets_tokenized.append([row[0], tweet])

    tweets_tokenized_df = pd.DataFrame(tweets_tokenized)
    #tweets_tokenized_df = tweets_tokenized_df.sample(frac=1).reset_index(drop=True)
    return tweets_tokenized_df

## 2.3. Przygotowanie zbioru uczącego

W pliku *train.csv* znajduje się zbiór treningowy, zawierający identyfikator tweet'a, jego treść oraz przyporządkowaną mu kategorię. Należy zwrócić uwagę, iż w zbiorze znajdują się tweet'y nie zawierające treści - etykietowane jako *Not Avaible*. Zostało to uwzględnione podczas wczytywania, ponieważ w innym wypadku etykieta zostałaby potraktowana jako treść tweet'a.

Wczytany zbiór treningowy jest losowo mieszany, co jest istotne z punktu widzenia późniejszego jego podziału do walidacji.

In [ ]:
tweets = pd.read_csv("data/train.csv", sep=',', header=0, na_values='Not Available')
tweets_tokenized_df = preprocess(tweets)
tweets_tokenized_df = tweets_tokenized_df.sample(frac=1).reset_index(drop=True)

# 3. Klasyfikator

Przeprowadzona w ramach projektu analiza wykazała, że najlepsze rezultaty otrzymano stosując klasyfikator *MultinomialNB* oraz reprezentacji TF-IDF.

In [ ]:
def NBayes_tfidf_classifier(tweets_tokenized_df, perc):
    from sklearn.pipeline import Pipeline
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.svm import LinearSVC
    from sklearn.naive_bayes import MultinomialNB

    data = tweets_tokenized_df.iloc[:,2].tolist()
    category = tweets_tokenized_df.iloc[:,1].tolist()
    num = int(round(len(data_tmp) * perc))
    
    data_tmp = []
    for row in data:
        if row == []:
            data_tmp.append('')
        else:
            data_tmp.append(' '.join(row))

    classifier = Pipeline([
        ('count_vectorizer', CountVectorizer(ngram_range=(1,  2), max_df=0.8, min_df=8)),
        ('tfidf_transformer', TfidfTransformer()),
        ('classifier', MultinomialNB())
    ])

    classifier.fit(data_tmp[:num], category[:num])

    print("Train set size ", num)
    print("Test set size ", len(data_tmp)-num)
    from sklearn.metrics import classification_report
    prediction = classifier.predict(data_tmp[:num])
    print("Train data classification report:")
    print(classification_report(category[:num], prediction))

    if len(data_tmp)-num > 0:
        print("Test data classification report:")
        prediction2 = classifier.predict(data_tmp[num:])
        print(classification_report(category[num:], prediction2))

    print("Submission set predicting")
    tweets_test = pd.read_csv("data/test.csv", sep=',', header=0, na_values='Not Available')
    tweets_tokenized_df = preprocess(tweets_test)

    data = tweets_tokenized_df.iloc[:,-1].tolist()
    data_tmp = []
    tt = 0;
    for row in data:
        if row == []:
            data_tmp.append('')
        else:
            data_tmp.append(' '.join(row))

    prediction3 = classifier.predict(data_tmp)

    submission = pd.concat([tweets_tokenized_df.iloc[:, 0], pd.DataFrame(prediction3)], axis=1)
    submission.columns = ['Id', 'Category']
    submission.to_csv('submission.csv', sep=',', index=False, index_label=False)

    print("Data saved to file")

Ostatecznie, wywołanie funkcji ma postać:

In [ ]:
NBayes_tfidf_classifier(tweets_tokenized_df, 0.7)